In [1]:
import os

In [2]:
%pwd

'/Users/manraj/Documents/GitHub/text-summariser-E2E-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/manraj/Documents/GitHub/text-summariser-E2E-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    DATA_PATH: Path
    TOKENIZER_NAME: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        return DataTransformationConfig(
            root_dir=self.config.data_transformation.root_dir,
            DATA_PATH=self.config.data_transformation.DATA_PATH,
            TOKENIZER_NAME=Path(self.config.data_transformation.TOKENIZER_NAME)
        )

In [8]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/Users/manraj/Documents/GitHub/text-summariser-E2E-project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/manraj/Documents/GitHub/text-summariser-E2E-project/venv/lib/python3.12/site-packages/ipykernel_launcher.py", 

[2024-12-19 18:58:19,398]: INFO: config: PyTorch version 2.2.2 available.


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.TOKENIZER_NAME)
        
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
        
    def convert(self):
        dataset_samsum = load_from_disk(self.config.DATA_PATH)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,'dataset_samsum_pt'))


In [10]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-12-19 18:58:20,005]: INFO: common: Successfully read YAML file: config/config.yaml
[2024-12-19 18:58:20,007]: INFO: common: Successfully read YAML file: params.yaml
[2024-12-19 18:58:20,008]: INFO: common: Created directory: artifacts
[2024-12-19 18:58:20,008]: INFO: common: Created directory: artifacts/data_transformation


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/manraj/Documents/GitHub/text-summariser-E2E-project/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 117092.95 examples/s]
